# **Building a Graph Neural Network**

This jupyter notebook build a Graph Neural Network (GNN) using PyTorch Geometric for prdiction of pIC50 for beta secretase 1 inhibitors.

Graph Neural Networks (GNNs) are a class of neural networks designed to operate on graph-structured data. Unlike traditional neural networks that process data in fixed shapes (like images or sequences), GNNs
can process data where relationships between entities are represented as nodes and edges.

In GNNs, the model learns to update node (atom) representations by aggregating information from neighboring nodes in the graph through a process called message passing. This allows GNNs to capture structural and
relational information from the graph, making them well-suited for tasks like molecular property prediction.

- **Nodes** represent atoms in a molecule and are characterized by atomic properties such as:
  - **Atomic number**
  - **Formal charge**
  - **Number of hydrogens**
  - **Aromaticity**

- **Edges** represent bonds between atoms and are characterized by bond properties such as:
  - **Bond type**
  - **Conjugation**

The target variable (pIC50) is used for predicting the bioactivity of drug molecules.

The GNN is trained to predict pIC50 values for each molecule by learning from its atomic structure and bond connectivity.



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#!pip install torch-scatter torch-sparse torch-geometric torch-cluster torch-spline-conv
!pip install torch-geometric

In [5]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 31.8 MB/s eta 0:00:00


In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold
import pandas as pd
from rdkit import Chem
import numpy as np


In [7]:
import torch
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data

# Sample node features and batch information
x = torch.randn(10, 64)  # 10 nodes with 64 features each
batch = torch.tensor([0, 0, 0, 0, 1, 1, 1, 2, 2, 2])  # 3 graphs with different numbers of nodes

# Perform global mean pooling
pooled_x = global_mean_pool(x, batch)
print(pooled_x)

tensor([[ 2.3312e-01,  4.2021e-01, -4.7386e-01,  1.5093e-01, -8.4034e-01,
          2.1865e-01,  1.0307e+00,  6.2408e-01, -7.4739e-01, -2.7835e-01,
         -1.3534e-03,  1.2982e+00, -5.7452e-03,  6.1854e-01, -3.0736e-01,
          7.3972e-01, -8.2827e-01, -7.0377e-01, -7.5256e-01, -4.4533e-01,
          1.0250e-01,  2.1168e-01, -4.3651e-01,  1.7993e-01,  9.6723e-01,
          4.1656e-01,  2.6120e-01, -3.6502e-01, -6.4667e-02,  1.4975e-01,
          1.1599e+00,  5.1622e-01, -7.4736e-01,  1.5449e-01,  3.5247e-01,
         -7.2597e-01,  3.4157e-01,  3.4300e-01,  1.8331e-02, -4.6809e-01,
          5.9859e-02,  2.8189e-01,  3.6192e-01,  2.8616e-02,  9.2539e-01,
         -3.9071e-01,  2.1226e-01,  6.4811e-01,  8.1833e-01, -1.9609e-01,
          1.0110e-01, -1.0267e+00,  5.3071e-01, -8.2641e-01, -1.9165e-01,
          4.3179e-02, -6.2312e-01, -3.3790e-01, -7.1961e-01, -2.4275e-01,
         -1.4789e-01,  3.7641e-01, -3.6507e-02,  2.9656e-01],
        [-3.6916e-01, -1.7916e-02,  1.5049e-02,  1

In [16]:
#Load dataset
file1 = "/content/drive/MyDrive/bioactivity/beta_secretase1_bioactivity_data_pIC50.csv"
df = pd.read_csv(file1)
df.head()

,molecule_chembl_id,bioactivity_class,standard_value,canonical_smiles,Molecular_Weight,LogP,HBD,HBA,pIC50
0,CHEMBL406146,intermediate,413.0,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@@H](N)CCC(=O...,999.085,-1.4355,13,13,6.384050
1,CHEMBL78946,active,2.0,CC(C)C[C@H](NC(=O)[C@H](CC(N)=O)NC(=O)[C@@H](N...,893.005,-1.7361,12,12,8.698970
2,CHEMBL324109,intermediate,460.0,CCC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(C)=O)[C@@H]...,751.988,2.3535,8,9,6.337242
3,CHEMBL114147,inactive,9000.0,CC(=O)NCC(=O)N[C@@H](Cc1ccccc1)[C@@H](O)CC(=O)...,737.895,1.9626,8,8,5.045757
4,CHEMBL419949,inactive,5600.0,CC(=O)N[C@@H](Cc1ccccc1)C(=O)N[C@@H](Cc1ccccc1...,828.020,3.5739,8,8,5.251812


In [17]:
# Extract SMILES and pIC50 from the DataFrame
smiles_list = df['canonical_smiles'].tolist()
pIC50_list = df['pIC50'].tolist()

# **Generating Graph Data**

In [18]:
# Function to generate graph data from SMILES
# This function converts the SMILES string into a graph representation (the input data X), and at the same time, it attaches the corresponding pIC50 value (the target y) to each graph.

def smiles_to_graph_extended(smiles, y):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None  # Handle invalid SMILES

    # Atom (node) features
    atom_features = []
    for atom in mol.GetAtoms():
        features = []
        features.append(atom.GetAtomicNum())  # Atomic number
        features.append(atom.GetFormalCharge())  # Formal charge
        features.append(atom.GetTotalNumHs())  # Total number of hydrogens
        features.append(atom.GetIsAromatic())  # Aromaticity
        atom_features.append(features)

    # Bond (edge) features and edge index
    edge_index = []
    edge_features = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append([i, j])
        edge_index.append([j, i])  # Both directions for undirected graph

        # Bond features (not used in this example)
        bond_feats = [bond.GetBondTypeAsDouble(), bond.GetIsConjugated()]
        edge_features.append(bond_feats)
        edge_features.append(bond_feats)  # Same for reverse direction

    # Convert to PyTorch tensors
    x = torch.tensor(atom_features, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Edge attributes are optional (using bond features here)
    edge_attr = torch.tensor(edge_features, dtype=torch.float)

    # Create PyTorch Geometric Data object (including target 'y' for pIC50)
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=torch.tensor([y], dtype=torch.float))
    return data


In [19]:
# Convert SMILES to graph format and generate graph data

graph_data_list = []
for smiles, pIC50 in zip(smiles_list, pIC50_list):
    graph_data = smiles_to_graph_extended(smiles, pIC50)
    if graph_data is not None:
        graph_data_list.append(graph_data)

In [ ]:
# Split data into training and test sets
#train_data, test_data = train_test_split(graph_data_list, test_size=0.2, random_state=42)

In [12]:
# Create PyTorch Geometric DataLoader for batch processing
#train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
#test_loader = DataLoader(test_data, batch_size=32)

## **Define Model**

In [20]:
class GNNModel(torch.nn.Module):
    def __init__(self, hidden_dim=64, num_layers=2):
        super(GNNModel, self).__init__()
        self.convs = torch.nn.ModuleList([GCNConv(4, hidden_dim)])  # First GCN layer
        self.convs.extend([GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers - 1)])  # Additional GCN layers
        self.fc1 = torch.nn.Linear(hidden_dim, 1)  # Fully connected layer: 64 input features to 1 output

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Apply GCN layers
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))

        # Global mean pooling over the nodes for each graph in the batch
        x = global_mean_pool(x, batch)  # Pooling each graph in the batch

        # Apply the fully connected layer to each graph representation
        x = self.fc1(x)

        return x.view(-1)  # Ensure output is flattened to match the batch size (one prediction per graph)





# **Model Training**

In [21]:
# Train the model
def train_model(model, train_loader, optimizer, criterion, max_epochs=100, patience=10):
    model.train()
    best_loss = np.inf
    patience_counter = 0
    for epoch in range(max_epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            out = model(batch)
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break
    return best_loss

# **Model Evaluation**

In [22]:
# Evaluate model
def evaluate_model(model, data_loader):
    model.eval()
    total_mse, total_mae = 0, 0
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch in data_loader:
            out = model(batch)

            # Convert batch predictions and actual values to NumPy arrays
            pred = out.view(-1).cpu().numpy()  # Flatten predictions to 1D array
            actual = batch.y.view(-1).cpu().numpy()  # Flatten actual values to 1D array

            # Append predictions and actuals for the entire batch
            predictions.extend(pred)
            actuals.extend(actual)

            # Calculate MSE and MAE for the entire batch
            mse = np.mean((pred - actual) ** 2)  # Mean squared error for the batch
            mae = np.mean(np.abs(pred - actual))  # Mean absolute error for the batch
            total_mse += mse * len(pred)  # Multiply by batch size to accumulate total
            total_mae += mae * len(pred)

    # Calculate final metrics (average over all batches)
    mse = total_mse / len(data_loader.dataset)
    mae = total_mae / len(data_loader.dataset)
    rmse = np.sqrt(mse)

    return mse, mae, rmse, predictions, actuals



In [23]:
# 5-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(kf.split(graph_data_list)):
    train_data = [graph_data_list[i] for i in train_index]
    test_data = [graph_data_list[i] for i in test_index]

    # Use DataLoader for batching
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    # Initialize the model, optimizer, and loss function
    model = GNNModel(hidden_dim=64, num_layers=2)  # You can adjust hidden_dim and num_layers
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Learning rate can also be tuned
    criterion = torch.nn.MSELoss()

    print(f"Training Fold {fold+1}...")
    # Train the model
    train_loss = train_model(model, train_loader, optimizer, criterion)

    # Evaluate the model on the train and test sets
    train_mse, train_mae, train_rmse, _, _ = evaluate_model(model, train_loader)
    test_mse, test_mae, test_rmse, _, _ = evaluate_model(model, test_loader)

    # Print metrics for the train and test sets
    print(f"Fold {fold+1} - Train MSE: {train_mse:.4f}, MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}")
    print(f"Fold {fold+1} - Test MSE: {test_mse:.4f}, MAE: {test_mae:.4f}, RMSE: {test_rmse:.4f}")

Training Fold 1...
Fold 1 - Train MSE: 1.3736, MAE: 0.9278, RMSE: 1.1720
Fold 1 - Test MSE: 1.3894, MAE: 0.9343, RMSE: 1.1787
Training Fold 2...
Fold 2 - Train MSE: 1.3514, MAE: 0.9305, RMSE: 1.1625
Fold 2 - Test MSE: 1.3527, MAE: 0.9373, RMSE: 1.1631
Training Fold 3...
Fold 3 - Train MSE: 1.4838, MAE: 0.9798, RMSE: 1.2181
Fold 3 - Test MSE: 1.4495, MAE: 0.9504, RMSE: 1.2039
Training Fold 4...
Fold 4 - Train MSE: 1.4540, MAE: 0.9526, RMSE: 1.2058
Fold 4 - Test MSE: 1.4827, MAE: 0.9760, RMSE: 1.2177
Training Fold 5...
Fold 5 - Train MSE: 1.4136, MAE: 0.9520, RMSE: 1.1889
Fold 5 - Test MSE: 1.4495, MAE: 0.9550, RMSE: 1.2040
